# MongoDB Atlas

This notebook covers how to MongoDB Atlas vector search in LangChain, using the `langchain-mongodb` package.

>[MongoDB Atlas](https://www.mongodb.com/docs/atlas/) is a fully-managed cloud database available in AWS, Azure, and GCP.  It supports native Vector Search and full text search (BM25) on your MongoDB document data.

>[MongoDB Atlas Vector Search](https://www.mongodb.com/products/platform/atlas-vector-search) allows to store your embeddings in MongoDB documents, create a vector search index, and perform KNN search with an approximate nearest neighbor algorithm (`Hierarchical Navigable Small Worlds`). It uses the [$vectorSearch MQL Stage](https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-overview/). 

## Setup

>*An Atlas cluster running MongoDB version 6.0.11, 7.0.2, or later (including RCs).

To use MongoDB Atlas, you must first deploy a cluster. We have a Forever-Free tier of clusters available. To get started head over to Atlas here: [quick start](https://www.mongodb.com/docs/atlas/getting-started/).

You'll need to install `langchain-mongodb` and `pymongo` to use this integration, and we will also download `langchain-openai` for this notebook in order to use it for our embeddings.

In [2]:
pip install -qU langchain-mongodb pymongo langchain-openai


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Credentials

For this notebook you will need to set an `OPENAI_API_KEY` and also find your MongoDB cluster URI.

For information on finding your cluster URI read through [this guide](https://www.mongodb.com/docs/manual/reference/connection-string/).

In [33]:
import getpass
import os

MONGODB_ATLAS_CLUSTER_URI = getpass.getpass("MongoDB Atlas Cluster URI:")
if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass.getpass("OpenAI API Key:")

## Instantiation

In [36]:
from pymongo import MongoClient
from langchain_openai import OpenAIEmbeddings
from langchain_mongodb.vectorstores import MongoDBAtlasVectorSearch
from pymongo import MongoClient

# initialize MongoDB python client
client = MongoClient(MONGODB_ATLAS_CLUSTER_URI)

DB_NAME = "langchain_test_db"
COLLECTION_NAME = "langchain_test_vectorstores"
ATLAS_VECTOR_SEARCH_INDEX_NAME = "langchain-test-index-vectorstores"

MONGODB_COLLECTION = client[DB_NAME][COLLECTION_NAME]
embedding_function = OpenAIEmbeddings()

vector_store = MongoDBAtlasVectorSearch(
  collection=MONGODB_COLLECTION,
  embedding=embedding_function,
  index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME,
  relevance_score_fn="cosine",
)

## Manage vector store

### Add items to vector store


In [39]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="foo",
    metadata={"source": "https://example.com"}
)

document_2 = Document(
    page_content="bar",
    metadata={"source": "https://example.com"}
)

document_3 = Document(
    page_content="baz",
    metadata={"source": "https://example.com"}
)

documents = [document_1, document_2, document_3]

vector_store.add_documents(documents=documents,ids=["1","2","3"])

['1', '2', '3']

### Delete items from vector store


In [40]:
vector_store.delete(ids=["3"])

True

## Query vector store

Once your vector store has been created and the relevant documents have been added you will most likely wish to query it during the running of your chain or agent. 

### Query directly

Performing a simple similarity search can be done as follows:

In [47]:
results = vector_store.similarity_search(query="foo",k=1)
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

* foo [{'_id': '1', 'source': 'https://example.com'}]


### Pre-filtering with Similarity Search

Atlas Vector Search supports pre-filtering using MQL Operators for filtering.  Below is an example index and query on the same data loaded above that allows you do metadata filtering on the "page" field.  You can update your existing index with the filter defined and do pre-filtering with vector search.

```json
{
  "fields":[
    {
      "type": "vector",
      "path": "embedding",
      "numDimensions": 1536,
      "similarity": "cosine"
    },
    {
      "type": "filter",
      "path": "source"
    }
  ]
}
```

You can also update the index programmatically using the `MongoDBAtlasVectorSearch.create_index` method.

```python
vectorstore.create_index(
  dimensions=1536,
  filters=[{"type":"filter", "path":"source"}],
  update=True
)
```

And then you can run a query with filter as follows:

```python
results = vector_store.similarity_search(query="foo",k=1,pre_filter={"source": {"$eq": "https://example.com"}})
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")
```

# Other Notes
>* More documentation can be found at [LangChain-MongoDB](https://www.mongodb.com/docs/atlas/atlas-vector-search/ai-integrations/langchain/) site
>* This feature is Generally Available and ready for production deployments.
>* The langchain version 0.0.305 ([release notes](https://github.com/langchain-ai/langchain/releases/tag/v0.0.305)) introduces the support for $vectorSearch MQL stage, which is available with MongoDB Atlas 6.0.11 and 7.0.2. Users utilizing earlier versions of MongoDB Atlas need to pin their LangChain version to <=0.0.304
> 

## API reference

For detailed documentation of all `MongoDBAtlasVectorSearch` features and configurations head to the API reference: https://api.python.langchain.com/en/latest/mongodb_api_reference.html